In [87]:
# 1
# Chance Wiese

In [89]:
# 2
import os
import pandas as pd

path = '/Users/chancewiese/Dropbox/5330corgis'
files = os.listdir(path)

for file in files:
    print(file)

102121-7380.csv
pdx623.csv
102121-5390.csv
portlandeventjun22.csv
newreg-062822.csv
102123-3428.csv
102124-9377.csv
Icon
newcorgis-20220619.csv
newcorgis-20220618.csv
signupsjune272022.csv
102124-3988.csv
102119-8469.csv
tacoma-062322.csv
102122-4015.csv
102120-8243.csv


In [91]:
# 3
import psycopg2

conn = psycopg2.connect(
    database='corgirace',
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)

cursor = conn.cursor()

query = "SELECT * FROM ingest;"
df_ingest = pd.read_sql_query(query, conn)

for file in files:
    file_path = os.path.join(path, file)
    file_lower = file.lower()
    
    if not file.endswith('.csv'):
        continue
    
    if file_lower in df_ingest['filename'].values:
        print(f"{file_lower} has already been processed.")
        continue
        
    with open(file_path, 'r') as newfile:
        next(newfile)
        print(f"Processing {file_lower}:")
        for line in newfile:
            issueslist = []
            
            line = line.strip()
            columns = line.split(',')
            
            if len(columns) > 5:
                issueslist.append("Too Many Columns")
            elif len(columns) < 5:
                issueslist.append("Too Few Columns")
            
            name = columns[0]
            if len(name) < 1:
                issueslist.append("Name Too Short")
            elif len(name) > 100:
                issueslist.append("Name Too Long")
            
            breed = columns[1]
            if not breed.upper().startswith(('P', 'C')):
                issueslist.append("Breed Issue")
                
            gender = columns[2]
            if gender.upper() not in ['M', 'F', 'SF', 'NM']:
                issueslist.append("Gender Col Issue")
                
            try:
                weight = float(columns[3])
                if weight < 15:
                    issueslist.append("Weight Too Low")
                elif weight > 35:
                    issueslist.append("Weight Too High")
            except ValueError:
                issueslist.append("Weight Non-Numeric")
                
            try:
                age = float(columns[4])
                if age < 1:
                    issueslist.append("Age Too Low")
                elif age > 10:
                    issueslist.append("Age Too High")
            except ValueError:
                issueslist.append("Age Non-Numeric")
                
            print(f"file: {file_lower}, name: {name if name else 'no_name'}, issues: {issueslist}")

102121-7380.csv has already been processed.
pdx623.csv has already been processed.
102121-5390.csv has already been processed.
portlandeventjun22.csv has already been processed.
newreg-062822.csv has already been processed.
102123-3428.csv has already been processed.
102124-9377.csv has already been processed.
newcorgis-20220619.csv has already been processed.
newcorgis-20220618.csv has already been processed.
signupsjune272022.csv has already been processed.
102124-3988.csv has already been processed.
102119-8469.csv has already been processed.
tacoma-062322.csv has already been processed.
102122-4015.csv has already been processed.
Processing 102120-8243.csv:
file: 102120-8243.csv, name: Megawoof, issues: ['Weight Too High', 'Age Too High']
file: 102120-8243.csv, name: Charles, issues: ['Gender Col Issue', 'Weight Too High']
file: 102120-8243.csv, name: Mabel the Claw, issues: []
file: 102120-8243.csv, name: Instant Breakfast, issues: []
file: 102120-8243.csv, name: Willow The Dust S

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_17427/739805483.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ingest = pd.read_sql_query(query, conn)


In [74]:
# 4
cursor.execute('''
CREATE TABLE IF NOT EXISTS corgi_exception (
    exceptid SERIAL PRIMARY KEY,
    except_record TEXT,
    origin_file VARCHAR(100) NOT NULL,
    issues TEXT,
    except_timestamp TIMESTAMP,
    fixed_timestamp TIMESTAMP
);
''')

conn.commit()

In [93]:
# 5
for file in files:
    file_path = os.path.join(path, file)
    file_lower = file.lower()
    
    if not file.endswith('.csv'):
        continue
       
    if file_lower in df_ingest['filename'].values:
        print(f"{file_lower} has already been processed.")
        continue
    
    with open(file_path, 'r') as newfile:
        next(newfile)
        
        for line in newfile:
            issueslist = []
            
            line = line.strip()
            columns = line.split(',')
            
            compliant = True
            
            if len(columns) != 5:
                compliant = False
                issueslist.append("Column Issue")
            else:
                name = columns[0].replace("'", "")
                if len(name) < 1 or len(name) > 100:
                    compliant = False
                    issueslist.append("Name Issue")
                
                breed = columns[1].lower()
                if breed.startswith('p'):
                    breed = 'Pem'
                elif breed.startswith('c'):
                    breed = 'Cardi'
                else:
                    compliant = False
                    issueslist.append("Breed Issue")
                
                gender = columns[2].upper()
                if gender not in ['M', 'F', 'NM', 'SF']:
                    compliant = False
                    issueslist.append("Gender Issue")
                
                try:
                    weight = float(columns[3])
                    if weight < 15 or weight > 35:
                        compliant = False
                        issueslist.append("Weight Issue")
                    else:
                        weight = round(weight)
                except ValueError:
                    compliant = False
                    issueslist.append("Weight Non-Numeric")
                
                try:
                    age = float(columns[4])
                    if age < 1 or age > 10:
                        compliant = False
                        issueslist.append("Age Issue")
                    else:
                        age = round(age * 2) / 2
                except ValueError:
                    compliant = False
                    issueslist.append("Age Non-Numeric")
            
            if compliant:
                cursor.execute(f'''
                    INSERT INTO corgi (corgname, breed, gender, weight, age, fromfile)
                    VALUES ('{name}', '{breed}', '{gender}', {weight}, {age}, '{file_lower}');
                ''')
                conn.commit()
                print(f"Record created for {name}")
            else:
                record_string = '|'.join(columns)
                issues_string = '|'.join(issueslist)
                
                cursor.execute(f'''
                    INSERT INTO corgi_exception 
                    (except_record, origin_file, issues, except_timestamp)
                    VALUES ('{record_string}', '{file_lower}', '{issues_string}', current_timestamp);
                ''')
                conn.commit()
                print(f"Record for {columns[0] if columns[0] else 'no_name'} had these issues: {issueslist}")
    
    cursor.execute(f'''
        INSERT INTO ingest (filename, whendone)
        VALUES ('{file_lower}', current_timestamp);
    ''')
    conn.commit()
    print(f"Ingestion complete for {file_lower}.")

102121-7380.csv has already been processed.
pdx623.csv has already been processed.
102121-5390.csv has already been processed.
portlandeventjun22.csv has already been processed.
newreg-062822.csv has already been processed.
102123-3428.csv has already been processed.
102124-9377.csv has already been processed.
newcorgis-20220619.csv has already been processed.
newcorgis-20220618.csv has already been processed.
signupsjune272022.csv has already been processed.
102124-3988.csv has already been processed.
102119-8469.csv has already been processed.
tacoma-062322.csv has already been processed.
102122-4015.csv has already been processed.
Record for Megawoof had these issues: ['Weight Issue', 'Age Issue']
Record for Charles had these issues: ['Gender Issue', 'Weight Issue']
Record created for Mabel the Claw
Record created for Instant Breakfast
Record created for Willow The Dust Storm Corgifriend
Record created for Empanada
Record created for Leatherface
Record created for Corgi Elliot
Recor

In [95]:
# 6
query = '''
SELECT exceptid, except_record, origin_file 
FROM corgi_exception 
ORDER BY except_record 
LIMIT 10;
'''

df_exceptions = pd.read_sql_query(query, conn, index_col='exceptid')
print("\ncorgi_exception table:\n", df_exceptions)


corgi_exception table:
                             except_record      origin_file
exceptid                                                  
37                 Amelia the Woof|||27|3  102120-8243.csv
12               Baby Congrats|PWC|M|36|4  102123-3428.csv
3         Bailey Barkbark|Cardi|SF|26|2||  102121-7380.csv
2                     Bandit|Pem|M|31|4||  102121-7380.csv
8                    Bowser|Pem|NM|27|4||  102121-7380.csv
13                       |Cardi|SF|26|4.5  102123-3428.csv
35                    Charles|Pem||36|2.5  102120-8243.csv
33                    Charles|Pem||36|2.5  102120-8243.csv
26                Crossbow Jo|Pem|R|5|2.7  102124-3988.csv
24         Double-Dragon|Cardi|NM|32|3.5|  102124-9377.csv


/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_17427/1888758769.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_exceptions = pd.read_sql_query(query, conn, index_col='exceptid')


In [117]:
# 7
query = '''
SELECT corgid, corgname, fromfile
FROM corgi c
JOIN ingest i ON c.fromfile = i.filename
WHERE EXTRACT(YEAR FROM whendone) = EXTRACT(YEAR FROM CURRENT_DATE)
AND EXTRACT(MONTH FROM whendone) = EXTRACT(MONTH FROM CURRENT_DATE)
ORDER BY corgname
LIMIT 10;
'''

df_corgi = pd.read_sql_query(query, conn, index_col='corgid')
print("\ncorgi table:\n", df_corgi)


corgi table:
                    corgname                fromfile
corgid                                             
43       Amber Waves of Fur  portlandeventjun22.csv
96                Baby Bear         102122-4015.csv
46             Bag o Donuts  portlandeventjun22.csv
59      Betty Doggy Sprinty       newreg-062822.csv
74               Briar Rose       tacoma-062322.csv
82             Brunchkiller         102123-3428.csv
85                Campstove         102123-3428.csv
37          CanCan of Beans              pdx623.csv
65               Cashew Nut   signupsjune272022.csv
73                Cleopupra       tacoma-062322.csv


/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_17427/1168531242.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_corgi = pd.read_sql_query(query, conn, index_col='corgid')


In [119]:
cursor.close()
conn.close()